In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.utils import resample
import torch
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.cuda.empty_cache()

In [ ]:
pd_orientation = pd.read_csv('drive/MyDrive/nlp-the2/orientation-tr-train.tsv', sep='\t')
train_data, test_data = train_test_split(
    pd_orientation,
    test_size=0.1,  # 10% for test
    random_state=42,  # Set seed for reproducibility
    stratify=pd_orientation['label']  # Ensure stratified split
)

In [ ]:
train_data, test_data = train_test_split(
    pd_orientation,
    test_size=0.15,  # 15% for test
    random_state=42,  # Set seed for reproducibility
    stratify=pd_orientation['label']  # Ensure stratified split
)

# Separate features and labels for training and testing
X_test = test_data['text_en'].tolist()
y_test = test_data['label'].tolist()

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
def generate_predictions_batched(speeches, tokenizer, model, batch_size=16):
    predictions = []

    for i in tqdm(range(0, len(speeches), batch_size)):
        batch = speeches[i:i + batch_size]
        prompts = [f"""Classify the following parliamentary speech as either:
        0: Left-leaning
        1: Right-leaning

        Speech: "{speech}"
        Answer with just the number 0 or 1.""" for speech in batch]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )

        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1,  # Limit response length
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.1,   # Reduce randomness
                do_sample=False
            )

        # Process batch predictions
        for output in outputs:
            pred_text = tokenizer.decode(output, skip_special_tokens=True)
            try:
                pred_label = int(''.join(filter(str.isdigit, pred_text.split()[-1])))
                if pred_label not in [0, 1]:
                    pred_label = None
            except (ValueError, IndexError):
                pred_label = None
            predictions.append(pred_label)

        torch.cuda.empty_cache()

    return predictions

In [ ]:
y_pred = generate_predictions_batched(X_test, tokenizer, model)

  0%|          | 0/152 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 152/152 [01:38<00:00,  1.54it/s]


In [ ]:
# Remove any None predictions for evaluation
valid_indices = [i for i, pred in enumerate(y_pred) if pred is not None]
valid_predictions = [y_pred[i] for i in valid_indices]
valid_y_test = [y_test.iloc[i] if hasattr(y_test, 'iloc') else y_test[i] for i in valid_indices]

# Evaluate
if len(valid_predictions) > 0:
    print("\nModel Performance:")
    print(f"Accuracy: {accuracy_score(valid_y_test, valid_predictions):.3f}")
    print("\nClassification Report:")
    print(classification_report(valid_y_test, valid_predictions,
                              target_names=['left-leaning', 'right-leaning']))

    print(f"\nPrediction Summary:")
    print(f"Total test samples: {len(y)}")
    print(f"Valid predictions: {len(valid_predictions)}")
    print(f"Invalid/None predictions: {y_pred.count(None)}")
else:
    print("No valid predictions were generated.")


Model Performance:
Accuracy: 0.592

Classification Report:
               precision    recall  f1-score   support

 left-leaning       0.00      0.00      0.00       443
right-leaning       0.59      1.00      0.74       643

     accuracy                           0.59      1086
    macro avg       0.30      0.50      0.37      1086
 weighted avg       0.35      0.59      0.44      1086


Prediction Summary:
Total test samples: 1878
Valid predictions: 1086
Invalid/None predictions: 1335


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
X_test_turkish = test_data['text'].tolist()
y_test = test_data['label'].tolist()

In [ ]:
def generate_predictions_batched_turkish(speeches, tokenizer, model, batch_size=16):
    predictions = []

    for i in tqdm(range(0, len(speeches), batch_size)):
        batch = speeches[i:i + batch_size]
        prompts = [f"""
        Aşağıdaki parlamento konuşmasını şu şekilde sınıflandırın:
        0: Sol eğilimli
        1: Sağ eğilimli

        Konuşma: "{speech}"
        Sadece 0 veya 1 sayısını cevap olarak verin.
        """ for speech in batch]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )

        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1,  # Limit response length
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.1,   # Reduce randomness
                do_sample=False
            )

        # Process batch predictions
        for output in outputs:
            pred_text = tokenizer.decode(output, skip_special_tokens=True)
            try:
                pred_label = int(''.join(filter(str.isdigit, pred_text.split()[-1])))
                if pred_label not in [0, 1]:
                    pred_label = None
            except (ValueError, IndexError):
                pred_label = None
            predictions.append(pred_label)

        torch.cuda.empty_cache()

    return predictions

In [ ]:
y_pred = generate_predictions_batched_turkish(X_test_turkish, tokenizer, model)

  0%|          | 0/152 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 152/152 [01:39<00:00,  1.53it/s]


In [ ]:
# Remove any None predictions for evaluation
valid_indices = [i for i, pred in enumerate(y_pred) if pred is not None]
valid_predictions = [y_pred[i] for i in valid_indices]
valid_y_test = [y_test.iloc[i] if hasattr(y_test, 'iloc') else y_test[i] for i in valid_indices]

# Evaluate
if len(valid_predictions) > 0:
    print("\nModel Performance:")
    print(f"Accuracy: {accuracy_score(valid_y_test, valid_predictions):.3f}")
    print("\nClassification Report:")
    print(classification_report(valid_y_test, valid_predictions,
                              target_names=['left-leaning', 'right-leaning']))

    print(f"\nPrediction Summary:")
    print(f"Total test samples: {len(y)}")
    print(f"Valid predictions: {len(valid_predictions)}")



Model Performance:
Accuracy: 0.419

Classification Report:
               precision    recall  f1-score   support

 left-leaning       0.42      0.97      0.59       219
right-leaning       0.22      0.01      0.01       292

     accuracy                           0.42       511
    macro avg       0.32      0.49      0.30       511
 weighted avg       0.31      0.42      0.26       511


Prediction Summary:
Total test samples: 1878
Valid predictions: 511
